In [5]:
import os
import cv2
import numpy as np

In [6]:

    
def load_data(data_dir):
    images = []
    labels = []
    
    for label in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, label)
        if os.path.isdir(folder_path):
            for img_name in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (64, 64)) 
                images.append(img)
                labels.append(label)
    
    return np.array(images), np.array(labels)

images, labels = load_data('data/')
images = images / 255.0  


In [7]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.3, random_state=132)


In [9]:
np.unique(labels)

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',
       'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], dtype='<U1')

In [22]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout with 50% rate
    Dense(len(np.unique(labels)), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the generator to your training data
datagen.fit(X_train)

# Now use datagen.flow() to train the model with augmented data
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=50, validation_data=(X_test, y_test))
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

e:\vs code\cv_project\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 82s 88ms/step - accuracy: 0.0319 - loss: 3.9510 - val_accuracy: 0.0260 - val_loss: 3.5561
Epoch 2/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 70s 76ms/step - accuracy: 0.0288 - loss: 3.5556 - val_accuracy: 0.0260 - val_loss: 3.5563
Epoch 3/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 67s 73ms/step - accuracy: 0.0292 - loss: 3.5555 - val_accuracy: 0.0260 - val_loss: 3.5563
Epoch 4/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 67s 73ms/step - accuracy: 0.0297 - loss: 3.5554 - val_accuracy: 0.0260 - val_loss: 3.5563
Epoch 5/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 68s 74ms/step - accuracy: 0.0306 - loss: 3.5552 - val_accuracy: 0.0260 - val_loss: 3.5563
Epoch 6/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 67s 73ms/step - accuracy: 0.0304 - loss: 3.5553 - val_accuracy: 0.0263 - val_loss: 3.5562
Epoch 7/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 67s 73ms/step - accuracy: 0.0298 - loss: 3.5555 - val_accuracy: 0.0263 - val_loss: 3.5562
Epoch 8/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 69s 75ms/step - accuracy: 0.0312 - loss: 3.5553 - 

In [34]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


394/394 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.0268 - loss: 3.5564
Test Accuracy: 2.60%


In [35]:
import pickle

In [40]:
filename = 'finalized_model_v0.3.sav'

In [41]:
pickle.dump(model, open(filename, 'wb'))

In [42]:
loaded_model = pickle.load(open(filename, 'rb'))

In [44]:
with open('finalized_model_v0.3.sav', 'rb') as file:
    model = pickle.load(file)

In [45]:
cap = cv2.VideoCapture(0)  

input_size = (64, 64) 

In [46]:
while True:
    ret, frame = cap.read()  #
    if not ret:
        break  
  
    resized_frame = cv2.resize(frame, input_size) 
    normalized_frame = resized_frame / 255.0  
    

    input_frame = np.expand_dims(normalized_frame, axis=0)
    

    prediction = model.predict(input_frame)
    
  
    predicted_class = np.argmax(prediction)
    
 
    label = f"Predicted Class: {predicted_class}"
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
 
    cv2.imshow('Webcam Prediction', frame)
    
    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━